# Purpose:
Watchlist Expiry has been deployed on all wikis. [T264249](https://phabricator.wikimedia.org/T264249) This notebook is to snapshot data of new feature adoption weekly. The data is the data source of [watchlist dashboard]( https://superset.wikimedia.org/r/351), used for Comm Tech team to estimate the adoption rate ( temporary watches / total watches) during the same time period.
 <br />
- Collect data of total temporary watches by wikis.
- Collect data of total watches since the fisrt recorder of temporary watches, including temporary watches and permanent watches, aggregated by wikis. <br />



# Feature deployment schedule: 
Deployed on September 22, 2020: Officewiki, Mediawikiwiki <br />
Deployed on September 29, 2020: Metawiki, Enwikisource, Enwikivoyage, Enwikiversity, Eswiktionary, Eswikisource, Eswikivoyage, Hewiki, Hewikisource, Hewiktionary, and Dewikisource. <br />
Deployed on October 13, 2020:  Dewiki, Frwiki, Fawiki, and Cswiki. <br />
Deployed on November 17, 2020: Wikimedia Commons and Wikidata. <br />
Deployed on December 1, 2020: Release to all remaining wikis. <br />


# Run frequency:
Weekly


In [1]:
import pandas as pd
from wmfdata import mariadb,hive

In [2]:
wikis =hive.run(
'''
SELECT database_code, database_group
FROM  canonical_data.wikis 
WHERE
    database_group in (
        "commons", "incubator", "foundation", "mediawiki", "meta", "sources", 
        "species","wikibooks", "wikidata", "wikinews", "wikipedia", "wikiquote",
        "wikisource", "wikiversity", "wikivoyage", "wiktionary"
    )
'''
).sort_values("database_code").set_index("database_code")

In [3]:
wikis.head()

,database_group
database_code,
aawiki,wikipedia
aawikibooks,wikibooks
aawiktionary,wiktionary
abwiki,wikipedia
abwiktionary,wiktionary


In [4]:
now = pd.Timestamp.utcnow()
today=now.date().strftime("%Y-%m-%d 00:00:00.000")

In [5]:
today

'2021-02-04 00:00:00.000'

In [6]:
columns = ['watch_type','user_type', 'wiki_db', 'snapshot','watches']
df_all= pd.DataFrame( columns=columns)

In [7]:
df_all

,watch_type,user_type,wiki_db,snapshot,watches


Command used to create table on hive:

CREATE TABLE IF NOT EXISTS jiawang.watchlist_snapshot (
watch_type STRING,
user_type STRING,
wiki_db STRING,
snapshot TIMESTAMP ,
watches BIGINT
) ROW FORMAT DELIMITED FIELDS TERMINATED BY "\t"

CREATE TABLE IF NOT EXISTS jiawang.watchlist_snapshot_latest (
watch_type STRING,
user_type STRING,
wiki_db STRING,
snapshot TIMESTAMP ,
watches BIGINT
) ROW FORMAT DELIMITED FIELDS TERMINATED BY "\t"

In [8]:
watch_count_query='''
SELECT CASE WHEN we.we_item IS NULL THEN 'permanent' ELSE 'temporary'   END AS watch_type,
(CASE WHEN (ug.ug_group = 'bot') OR (u.user_name regexp "bot\\b") THEN 'bot' ELSE 'nonbot' END) AS user_type,
'{wiki}' AS wiki_db, '{date}' AS snapshot,
COUNT(wl.wl_id) AS watches
FROM watchlist  AS wl 
LEFT JOIN watchlist_expiry AS we  ON we.we_item=wl.wl_id
INNER JOIN user AS u ON u.user_id = wl.wl_user
LEFT JOIN 
user_groups AS ug ON ug.ug_user=wl.wl_user  AND ug_group= 'bot'
WHERE wl.wl_id >= 
(SELECT MIN(we2.we_item) FROM watchlist_expiry  AS we2 )
GROUP BY
CASE WHEN we.we_item IS NULL THEN 'temporary' ELSE 'permanent' END,
(CASE WHEN (ug.ug_group = 'bot') OR (u.user_name regexp "bot\\b") THEN 'bot' ELSE 'nonbot' END)
'''

In [9]:
for w in wikis.index.values:
    print(w)
    try:
        df_tmp=mariadb.run(watch_count_query.format(wiki=w,date=today), w)
        df_all=df_all.append(df_tmp,ignore_index=True)
    except ValueError:
        pass

aawiki
aawikibooks
aawiktionary
abwiki
abwiktionary
acewiki
adywiki
afwiki
afwikibooks
afwikiquote
afwiktionary
akwiki
akwikibooks
akwiktionary
alswiki
amwiki
amwikiquote
amwiktionary
angwiki
angwikibooks
angwikiquote
angwikisource
angwiktionary
anwiki
anwiktionary
arcwiki
arwiki
arwikibooks
arwikinews
arwikiquote
arwikisource
arwikiversity
arwiktionary
arywiki
arzwiki
astwiki
astwikibooks
astwikiquote
astwiktionary
aswiki
aswikibooks
aswikisource
aswiktionary
atjwiki
avkwiki
avwiki
avwiktionary
awawiki
aywiki
aywikibooks
aywiktionary
azbwiki
azwiki
azwikibooks
azwikiquote
azwikisource
azwiktionary
banwiki
barwiki
bat_smgwiki
bawiki
bawikibooks
bclwiki
be_x_oldwiki
bewiki
bewikibooks
bewikiquote
bewikisource
bewiktionary
bgwiki
bgwikibooks
bgwikinews
bgwikiquote
bgwikisource
bgwiktionary
bhwiki
bhwiktionary
biwiki
biwikibooks
biwiktionary
bjnwiki
bmwiki
bmwikibooks
bmwikiquote
bmwiktionary
bnwiki
bnwikibooks
bnwikisource
bnwikivoyage
bnwiktionary
bowiki
bowikibooks
bowiktionary
bpywiki

twwiki
twwiktionary
tyvwiki
tywiki
udmwiki
ugwiki
ugwikibooks
ugwikiquote
ugwiktionary
ukwiki
ukwikibooks
ukwikinews
ukwikiquote
ukwikisource
ukwikivoyage
ukwiktionary
urwiki
urwikibooks
urwikiquote
urwiktionary
uzwiki
uzwikibooks
uzwikiquote
uzwiktionary
vecwiki
vecwikisource
vecwiktionary
vepwiki
vewiki
viwiki
viwikibooks
viwikiquote
viwikisource
viwikivoyage
viwiktionary
vlswiki
vowiki
vowikibooks
vowikiquote
vowiktionary
warwiki
wawiki
wawikibooks
wawiktionary
wikidatawiki
wowiki
wowikiquote
wowiktionary
wuuwiki
xalwiki
xhwiki
xhwikibooks
xhwiktionary
xmfwiki
yiwiki
yiwikisource
yiwiktionary
yowiki
yowikibooks
yowiktionary
zawiki
zawikibooks
zawikiquote
zawiktionary
zeawiki
zh_classicalwiki
zh_min_nanwiki
zh_min_nanwikibooks
zh_min_nanwikiquote
zh_min_nanwikisource
zh_min_nanwiktionary
zh_yuewiki
zhwiki
zhwikibooks
zhwikinews
zhwikiquote
zhwikisource
zhwikiversity
zhwikivoyage
zhwiktionary
zuwiki
zuwikibooks
zuwiktionary


In [10]:
df_all.tail()

,watch_type,user_type,wiki_db,snapshot,watches
423,temporary,nonbot,zhwikisource,2021-02-04 00:00:00.000,2
424,permanent,bot,zhwikisource,2021-02-04 00:00:00.000,10
425,permanent,nonbot,zhwikisource,2021-02-04 00:00:00.000,6084
426,temporary,nonbot,zuwiki,2021-02-04 00:00:00.000,2
427,permanent,nonbot,zuwiki,2021-02-04 00:00:00.000,1004


**Download data as tsv file.**

In [20]:
tsv_output_file='watchlist_snapshot_'+today[0:10]+'.tsv'

In [21]:
tsv_output_file

'watchlist_snapshot_2020-11-20.tsv'

In [22]:
df_all.to_csv(tsv_output_file, sep = '\t',
                 columns = ['watch_type', 'user_type','wiki_db', 'snapshot',
                            'watches'],
                 header = False, index = False)

**Command to upload data to hive:**

LOAD DATA LOCAL INPATH "/home/jiawang/src/CommTech/datasets/watchlist_snapshot_2021-01-xx.tsv"
OVERWRITE INTO TABLE jiawang.watchlist_snapshot_latest;

insert into table jiawang.watchlist_snapshot select * from jiawang.watchlist_snapshot_latest;